In [ ]:
from pprint import pprint 
from elasticsearch import Elasticsearch 
es = Elasticsearch('http://localhost:9200',basic_auth=('user','password')) 
client_info = es.info() 
print(' Connected to Elasticsearch!') 
pprint(client_info.body) 

 Connected to Elasticsearch!
{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'YkeBIv4WSkuNojcRv-HDJQ',
 'name': 'a15d97879705',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2025-06-18T22:09:56.772581489Z',
             'build_flavor': 'default',
             'build_hash': 'cc7302afc8499e83262ba2ceaa96451681f0609d',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '10.1.0',
             'minimum_index_compatibility_version': '8.0.0',
             'minimum_wire_compatibility_version': '8.18.0',
             'number': '9.0.3'}}


In [21]:
es.indices.delete(index='transcription',ignore_unavailable=True)
es.indices.create(
    index="transcription",
    settings={
        "index":{
            "number_of_shards":3,
            "number_of_replicas":2
        }
    },
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'transcription'})

In [22]:
import json
import os
from glob import glob
from tqdm import tqdm
def insert_docs(doc):
    response=es.index(index='transcription',body=doc)
    return response

def print_info(res):
    print(f"""
Document Id: {res['_id']} is {res['result']} and is split into {res['_shards']['total']} shreds
          """)
    
docs_ids=[]

def process_transcription_file(file_path):
    """Process a single transcription file and insert all segments"""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        # print(f"Processing file: {os.path.basename(file_path)}")
        
        # Insert each segment as a separate document
        for i, segment in enumerate(tqdm(data['segment'],desc="Segments", leave=False)):
            doc = {
                'video_link': data['video_link'],
                'segment_index': i,
                'start': segment['start'],
                'end': segment['end'],
                'text': segment['text']
            }
            
            res = insert_docs(doc)
            docs_ids.append(res['_id'])
        
        
    except Exception as e:
        print(f"❌ Error processing {file_path}: {e}\n")

# Method 1: Process all JSON files in a directory
transcription_dir = "../transcriptions_1445h_v3 - Copy"
json_files = glob(os.path.join(transcription_dir, "*.json"))

print(f"Found {len(json_files)} JSON files to process\n")

for file_path in json_files:
    process_transcription_file(file_path)

print("🎉 All files processed!")

Found 49 JSON files to process



🎉 All files processed!


In [10]:
#this is to show the datatypes of 
from pprint import pprint
index_mapping=es.indices.get_mapping(index='transcription')
pprint(index_mapping["transcription"]["mappings"]["properties"])

{'end': {'type': 'float'},
 'segment_index': {'type': 'long'},
 'start': {'type': 'float'},
 'text': {'fields': {'keyword': {'ignore_above': 256, 'type': 'keyword'}},
          'type': 'text'},
 'video_link': {'fields': {'keyword': {'ignore_above': 256, 'type': 'keyword'}},
                'type': 'text'}}


In [23]:
# count documents
es.count(index='transcription')

ObjectApiResponse({'count': 7250, '_shards': {'total': 3, 'successful': 3, 'skipped': 0, 'failed': 0}})

In [ ]:
#get a document
es.get(index='transcription',id=docs_ids[0])

ObjectApiResponse({'_index': 'transcription', '_id': '3tjeZ5kB4dmQKG5PqRxM', '_version': 1, '_seq_no': 2341, '_primary_term': 1, 'found': True, '_source': {'video_link': '0T28iTJFhxw', 'segment_index': 0, 'start': 0.0, 'end': 21.14, 'text': 'بسم الله الرحمن الرحيم الحمد لله والصلاة والسلام على رسول الله وعلى آله وصحبه ومن ولاه السلام عليكم جميعا ورحمة الله وبركاته وأهلا ومرحبا بكم في حلقة أخرى من هذا البرنامج وهذه الحلقات التي خصصناها للغة الإشارة لهذه الفئة الراقية كما تحدثنا بالأمس'}})

In [26]:
# get doucment with query
query={
    "match":{
        "text":"مرض"
    }
}

es.count(index='transcription',query=query)

ObjectApiResponse({'count': 16, '_shards': {'total': 3, 'successful': 3, 'skipped': 0, 'failed': 0}})

In [29]:
# serch doucment with match word
query={
    "match":{
        "text":"مرض"
    }
}

# Execute search and get results
response = es.search(index='transcription', query=query, size=10)
# Print results
for hit in response['hits']['hits']:
    print(f"Video: {hit['_source']['video_link']}")
    print(f"Time: {hit['_source']['start']} - {hit['_source']['end']}")
    print(f"Text: {hit['_source']['text']}")
    print("-" * 50)

Video: SGoFOmp2CYo
Time: 1781.66 - 1800.66
Text: من رحمة الله تبارك و تعالى فعلى من وقع في هذا الفعل إن وجدت ريبة لتفشي مرض في مجتمع من المجتمعات مرض نقص المناعة أو الإصابة بشيء من الأمراض الجنسية
--------------------------------------------------
Video: b8FezHrM5p8
Time: 2490.8 - 2506.38
Text: نعم و هذا يعني مرض نعم هو مرض قد يرجع إلى خلل جيني أو إلى أي سبب آخر و المرض يعالج نعم لكن الذي يتحدث عنه العالم اليوم هذا العالم كما قلت
--------------------------------------------------
Video: f0cKI3WBXpE
Time: 2497.14 - 2512.78
Text: ما كان بسبب علة هل كان ذلك بسبب مرض أصابه أو قرح في أسفل الركبة أو بسبب مرض آخر أو بسبب الموضع و طبيعة الأرض
--------------------------------------------------
Video: 0T28iTJFhxw
Time: 1927.08 - 1945.5
Text: قال فلا تخضعن بالقول فيطمع الذي في قلبه مرض وقلن قولا معروفا فطريقة الحديث لا بد أن لا يكون فيها خضوع وخناء ثم الحديث نفسه أن يكون من القول المعروف
--------------------------------------------------
Video: 6U1P6dnCjXA
Time: 417.56 - 432.7
Text: لغير ذوي الأع

In [30]:
# search document with match word
query={
    "match":{
        "text":"مرض"
    }
}

# Execute search and get results
response = es.search(index='transcription', query=query, size=10)

# Print results
for i, hit in enumerate(response['hits']['hits']):
    video_link = hit['_source']['video_link']
    start_time = hit['_source']['start']
    
    # Create YouTube link with timestamp
    if 'youtube.com/watch?v=' in video_link or 'youtu.be/' in video_link:
        # Extract video ID
        if 'youtube.com/watch?v=' in video_link:
            video_id = video_link.split('v=')[1].split('&')[0]
        else:  # youtu.be format
            video_id = video_link.split('youtu.be/')[-1].split('?')[0]
        
        # Create timestamped YouTube link
        youtube_link_with_time = f"https://www.youtube.com/watch?v={video_id}&t={int(start_time)}s"
    else:
        youtube_link_with_time = f"{video_link}&t={int(start_time)}s"
    
    print(f"Result {i+1}:")
    print(f"Video: {video_link}")
    print(f"YouTube Link with Time: {youtube_link_with_time}")
    print(f"Time: {hit['_source']['start']} - {hit['_source']['end']}")
    print(f"Text: {hit['_source']['text']}")
    print("-" * 50)

# If you want just the first result with timestamp:
if response['hits']['hits']:
    first_hit = response['hits']['hits'][0]
    video_link = first_hit['_source']['video_link']
    start_time = int(first_hit['_source']['start'])
    
    # Extract video ID and create timestamped link
    if 'youtube.com/watch?v=' in video_link:
        video_id = video_link.split('v=')[1].split('&')[0]
    elif 'youtu.be/' in video_link:
        video_id = video_link.split('youtu.be/')[-1].split('?')[0]
    else:
        video_id = video_link  # Assume it's already a video ID
    
    timestamped_link = f"https://www.youtube.com/watch?v={video_id}&t={start_time}s"
    
    print(f"\n🎯 First Result with Timestamp:")
    print(f"Link: {timestamped_link}")

Result 1:
Video: SGoFOmp2CYo
YouTube Link with Time: SGoFOmp2CYo&t=1781s
Time: 1781.66 - 1800.66
Text: من رحمة الله تبارك و تعالى فعلى من وقع في هذا الفعل إن وجدت ريبة لتفشي مرض في مجتمع من المجتمعات مرض نقص المناعة أو الإصابة بشيء من الأمراض الجنسية
--------------------------------------------------
Result 2:
Video: b8FezHrM5p8
YouTube Link with Time: b8FezHrM5p8&t=2490s
Time: 2490.8 - 2506.38
Text: نعم و هذا يعني مرض نعم هو مرض قد يرجع إلى خلل جيني أو إلى أي سبب آخر و المرض يعالج نعم لكن الذي يتحدث عنه العالم اليوم هذا العالم كما قلت
--------------------------------------------------
Result 3:
Video: f0cKI3WBXpE
YouTube Link with Time: f0cKI3WBXpE&t=2497s
Time: 2497.14 - 2512.78
Text: ما كان بسبب علة هل كان ذلك بسبب مرض أصابه أو قرح في أسفل الركبة أو بسبب مرض آخر أو بسبب الموضع و طبيعة الأرض
--------------------------------------------------
Result 4:
Video: 0T28iTJFhxw
YouTube Link with Time: 0T28iTJFhxw&t=1927s
Time: 1927.08 - 1945.5
Text: قال فلا تخضعن بالقول فيطمع الذي في قلبه مر